# ADPA Security Features

This notebook demonstrates the security features of the ADPA framework, including:
- Query validation and sanitization
- Access control and permissions
- Audit logging
- Secure data handling

## Setup

First, let's import the necessary components and set up our security configuration.

In [ ]:
from adpa import Text2SQL, Database
from adpa.config import Config
from adpa.security import SecurityManager, AccessControl, AuditLogger
from adpa.monitoring import Monitor

# Initialize components
config = Config()
config.load_env()

security = SecurityManager(config)
access_control = AccessControl(config)
audit = AuditLogger(config)
monitor = Monitor(config)

## Query Validation and Sanitization

Demonstrate how ADPA validates and sanitizes queries to prevent SQL injection and other security issues.

In [ ]:
# Example queries with potential security issues
queries = [
    "Show all employees",  # Safe query
    "SELECT * FROM users WHERE id = 1; DROP TABLE users;",  # SQL injection attempt
    "TRUNCATE TABLE employees",  # Destructive operation
    "SELECT * FROM pg_shadow"  # System table access attempt
]

text2sql = Text2SQL(config)

for query in queries:
    print(f"\nAnalyzing query: {query}")
    
    # Validate query
    validation = security.validate_query(query)
    print(f"Validation result: {validation}")
    
    if validation.is_valid:
        # Sanitize and execute query
        safe_query = security.sanitize_query(query)
        print(f"Sanitized query: {safe_query}")
        
        # Log the attempt
        audit.log_query_execution(safe_query)
    else:
        print(f"Security violations: {validation.violations}")
        # Log the security violation
        audit.log_security_violation(query, validation.violations)

## Access Control and Permissions

Show how ADPA manages access control and permissions.

In [ ]:
# Define user roles and permissions
roles = {
    'analyst': ['READ'],
    'manager': ['READ', 'WRITE'],
    'admin': ['READ', 'WRITE', 'DELETE']
}

# Test access control for different roles
test_cases = [
    ('analyst', 'SELECT * FROM employees'),
    ('analyst', 'UPDATE employees SET salary = 100000'),
    ('manager', 'UPDATE employees SET department = "Sales"'),
    ('admin', 'DELETE FROM employees WHERE id = 1')
]

for role, query in test_cases:
    print(f"\nTesting {role} access for query: {query}")
    
    # Check permissions
    if access_control.check_permission(role, query):
        print(f"Access granted for {role}")
        # Execute query with role context
        with access_control.role_context(role):
            result = text2sql.execute_secure(query)
            print(f"Query executed successfully")
    else:
        print(f"Access denied for {role}")
        # Log access violation
        audit.log_access_violation(role, query)

## Audit Logging

Demonstrate ADPA's audit logging capabilities.

In [ ]:
# Enable detailed audit logging
audit.set_level('DEBUG')

# Perform some sensitive operations
with audit.session() as audit_session:
    # Attempt sensitive data access
    query = "SELECT salary FROM employees"
    if security.is_sensitive_data(query):
        audit.log_sensitive_access(query)
        
    # Attempt schema modification
    query = "ALTER TABLE employees ADD COLUMN address TEXT"
    if security.is_schema_modification(query):
        audit.log_schema_modification(query)
        
    # Check audit trail
    print("\nAudit Trail:")
    for entry in audit.get_recent_entries():
        print(f"[{entry.timestamp}] {entry.action}: {entry.details}")

## Secure Data Handling

Show how ADPA handles sensitive data securely.

In [ ]:
# Define sensitive data patterns
security.add_sensitive_pattern(
    name="credit_card",
    pattern=r"\d{4}-\d{4}-\d{4}-\d{4}"
)

security.add_sensitive_pattern(
    name="ssn",
    pattern=r"\d{3}-\d{2}-\d{4}"
)

# Test data handling
test_data = [
    "John Doe, 123-45-6789",  # SSN
    "Card: 1234-5678-9012-3456",  # Credit card
    "Regular data: ABC123"
]

for data in test_data:
    print(f"\nProcessing: {data}")
    
    # Check for sensitive data
    if security.contains_sensitive_data(data):
        print("Sensitive data detected!")
        # Redact sensitive information
        safe_data = security.redact_sensitive_data(data)
        print(f"Redacted: {safe_data}")
        
        # Log the handling of sensitive data
        audit.log_sensitive_data_handling()
    else:
        print("No sensitive data detected")

## Cleanup

Clean up resources and generate security report.

In [ ]:
# Generate security report
report = security.generate_report()
print("\nSecurity Report:")
print(report)

# Clean up
audit.close()
monitor.shutdown()